## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Burnie,AU,-41.0667,145.9167,64.00,67,52,5.99,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,39.20,87,75,16.11,broken clouds
2,2,Lebu,CL,-37.6167,-73.6500,54.30,91,93,4.47,overcast clouds
3,3,Kearney,US,40.6995,-99.0815,55.40,87,1,4.61,clear sky
4,4,Tezu,IN,27.9167,96.1667,94.39,19,65,4.76,broken clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
8,8,Bathsheba,BB,13.2167,-59.5167,78.80,65,40,19.57,scattered clouds
10,10,Saint-Philippe,RE,-21.3585,55.7679,77.00,78,0,19.57,light rain
11,11,Avarua,CK,-21.2078,-159.7750,84.20,74,40,6.91,scattered clouds
12,12,Takoradi,GH,4.8845,-1.7554,79.79,85,100,5.41,overcast clouds
13,13,Rikitea,PF,-23.1203,-134.9692,75.72,65,84,10.56,broken clouds
14,14,Bhatkal,IN,13.9667,74.5667,85.30,69,19,2.42,few clouds
16,16,Arlit,NE,18.7369,7.3853,86.99,11,2,7.14,clear sky
18,18,Bengkulu,ID,-3.8004,102.2655,83.50,69,87,2.51,overcast clouds
28,28,Airai,TL,-8.9266,125.4092,76.28,40,55,2.33,broken clouds
30,30,Dingle,PH,10.9995,122.6711,88.61,55,48,6.55,light rain


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                191
City                   191
Country                191
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Weather Description    191
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
8,Bathsheba,BB,78.80,scattered clouds,13.2167,-59.5167,
10,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
11,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,
12,Takoradi,GH,79.79,overcast clouds,4.8845,-1.7554,
13,Rikitea,PF,75.72,broken clouds,-23.1203,-134.9692,
14,Bhatkal,IN,85.30,few clouds,13.9667,74.5667,
16,Arlit,NE,86.99,clear sky,18.7369,7.3853,
18,Bengkulu,ID,83.50,overcast clouds,-3.8004,102.2655,
28,Airai,TL,76.28,broken clouds,-8.9266,125.4092,
30,Dingle,PH,88.61,light rain,10.9995,122.6711,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
8,Bathsheba,BB,78.80,scattered clouds,13.2167,-59.5167,Atlantis Hotel
10,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,Paradise Inn
12,Takoradi,GH,79.79,overcast clouds,4.8845,-1.7554,Raybow International Hotel
13,Rikitea,PF,75.72,broken clouds,-23.1203,-134.9692,Pension Maro'i


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))